In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# mongoDB 사용을 위한 라이브러리
from pymongo import MongoClient
from datetime import datetime, timedelta

from django.shortcuts import render
from django.http import JsonResponse, FileResponse, HttpResponse

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document
from langchain.tools.retriever import create_retriever_tool
import re

In [4]:
load_dotenv()

True

In [5]:
api_key = os.getenv('OPENAI_API_KEY_sesac')


In [6]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key)


In [7]:
cluster = MongoClient(os.environ.get("mongo")) # 클러스터 
db = cluster['Document'] # 유저 정보
newsdata = db["newsdata"] # 내가 지정한 컬렉션 이름

In [8]:
newsdatas = newsdata.find().to_list()

docs = [Document(page_content = news_info['news_content'],metadata = {
                                                                        'title': news_info['news_title'],
                                                                        'first_time': news_info['news_first_upload_time'],
                                                                        'last_time':news_info['news_last_upload_time'],
                                                                        'subcategory':news_info['news_category'],
                                                                        'major category':news_info['note'],
                                                                        'website':news_info['news_website'],
                                                                        'url':news_info['news_url'],
                                                                        'author':news_info['news_author'],
                                                                        })
    for news_info in newsdatas]

In [9]:
splitter = CharacterTextSplitter(chunk_size = 50, chunk_overlap = 5)
split_texts = splitter.split_documents(docs)

Created a chunk of size 199, which is longer than the specified 50
Created a chunk of size 357, which is longer than the specified 50
Created a chunk of size 881, which is longer than the specified 50
Created a chunk of size 1099, which is longer than the specified 50
Created a chunk of size 488, which is longer than the specified 50
Created a chunk of size 1648, which is longer than the specified 50
Created a chunk of size 1352, which is longer than the specified 50
Created a chunk of size 2016, which is longer than the specified 50
Created a chunk of size 1681, which is longer than the specified 50
Created a chunk of size 419, which is longer than the specified 50
Created a chunk of size 750, which is longer than the specified 50
Created a chunk of size 201, which is longer than the specified 50
Created a chunk of size 176, which is longer than the specified 50
Created a chunk of size 432, which is longer than the specified 50
Created a chunk of size 337, which is longer than the spe

In [10]:
news_vector_store = FAISS.from_documents(split_texts, embeddings)
news_vector_store.save_local(rf'C:\Users\RMARKET\Desktop\mix_final\finance_agent_server\myapp\vecterDB',index_name = 'NewsData')